In [1]:
import pandas as pd
import numpy as np
import gensim

from gensim.models import Word2Vec

In [2]:
# Treinado na versão 4.2.0
print(gensim.__version__)

4.2.0


In [3]:
!ls '../data/'

D_p-pts.pickle	exp1-trj.h5	exp1-trj.t   README.md	   train.trg  vocab.txt
D_q-pts.pickle	exp1-trj.label	exp2-trj.h5  saved_models  val.src
Dq-pts.pickle	exp1-trj.pts	porto.csv    train.src	   val.trg


## Treino do modelo Word2Vec

In [4]:
# DISCRETIZAÇÃO DE CELS DO T2VEC:
trajs_treino = []
with open('../data/train.src', 'r') as f:
    for linha in f:
        # Divida as cels da linha usando espaço como delimitador e as converte em strings
        traj = linha.strip().split(' ')
        # Adicione a traj da linha à lista principal
        trajs_treino.append(traj)

In [5]:
print("Quantidade de trajetórias de treino:", len(trajs_treino))

Quantidade de trajetórias de treino: 13369584


In [6]:
print(trajs_treino[0])

['506', '112', '144', '148', '250', '258', '384', '106', '15', '4', '71', '1179', '93', '165', '160', '211', '300', '1245', '547']


In [7]:
%%time
"""
Training algorithm: Skip-gram or CBOW
Parameter --> sg: {0, 1} (1 for Skip-gram, 0 for CBOW)
"""
# Inicializando o modelo e treinando-o com 1 época:
model = Word2Vec(trajs_treino, vector_size=256, window=5, min_count=1, sg=1, epochs=1, alpha=0.025)

CPU times: user 31min 34s, sys: 939 ms, total: 31min 35s
Wall time: 14min 22s


In [8]:
# Definindo os critérios de parada:
max_epochs = 100
min_loss = 5000
previous_loss = float('inf')

In [9]:
# Definindo os critérios de parada:
max_epochs = 100
min_loss = 5000
previous_loss = float('inf')

In [10]:
%%time
# Treinando o modelo...
for epoch in range(2, max_epochs): # Comeca da 2a epoca, pois na inicializacao o modelo treina a 1a epoca
    model.train(trajs_treino, total_examples=len(trajs_treino), epochs=1, compute_loss=True)
    loss = model.get_latest_training_loss()

    # Verifique o critério de parada
    print('Dif. Loss:', previous_loss - loss)
    if (previous_loss - loss) < min_loss:
        print(f"Parando o treinamento após {epoch} épocas.")
        break

    previous_loss = loss

Dif. Loss: inf
Dif. Loss: -245616.0
Parando o treinamento após 3 épocas.
CPU times: user 59min 7s, sys: 1.49 s, total: 59min 9s
Wall time: 19min 47s


In [11]:
# Salvando o modelo...
model.save('../data/saved_models/W2V/w2v_w{}_s{}_ep{}_sg{}.model'.format(model.window, model.vector_size, epoch, model.sg))
print('w2v_w{}_s{}_ep{}_sg{}.model saved!'.format(model.window, model.vector_size, epoch, model.sg))

w2v_w5_s256_ep3_sg1.model saved!
